In [13]:
!pwd

/home/b/b382291/regridder/AQUA


In [4]:
# run mean OPA 
import intake 
import os
# add directory where the utils.py file is located
import xarray as xr 
import importlib
import dask as dk
import numpy as np 

# for levante
os.chdir('/home/b/b382291/git/one_pass/src')
imported_module = importlib.import_module("opa")

os.chdir('/home/b/b382291/git/one_pass/src')
importlib.reload(imported_module)
from opa import *
from opa import opa # from module (file.py) import class name 

os.chdir('/home/b/b382291/regridder/AQUA')
from aqua import Reader
from aqua.reader import catalogue


# initalising the statistic 
stat1 = opa(statistic = "min", statFreq = "daily", outputFreq = "weekly", save = "false")

# list of possible statistics: mean, var, std, min, max, threshExceed, hist, percentile



Running the algorithm

In [7]:

for i in range(0, 24, 1): 

    # extract moving window 
    ds = tasr.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    
    dm = stat1.compute(ds)



ValueError: Boolean index assignment in Dask expects equally shaped arrays.
Example: da1[da2] = da3 where da1.shape == (4,), da2.shape == (4,) and da3.shape == (4,).
Alternatively, you can use the extended API that supportsindexing with tuples.
Example: da1[(da2,)] = da3.

In [6]:
# from dask.distributed import LocalCluster, Client, progress
# processes = False
# nworker = 32
# ncpu = 32
# threads = ncpu // nworker
# print(f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}")
# client = Client(processes=processes,
#  threads_per_worker=threads,
#  n_workers=nworker,
#  memory_limit='64GB'
#  )

#catalogue();

# REGRIDDER 

os.chdir('/home/b/b382291/regridder/AQUA')

reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", regrid="r020")
data = reader.retrieve(fix=False)
tasr = reader.regrid(data["10u"][:,:])

#tasr = reader.regrid(data["2t"][0:16,:])

In [54]:
# checking the output against np.mean 

ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npStat = np.mean(ds2, axis = axNum, dtype = np.float64)
npStat = np.var(ds2, axis = axNum, dtype = np.float64, ddof = 1)
npStat = np.min(ds2, axis = axNum)
npStat = np.squeeze(npStat)

In [62]:
#a= stat1.minCum - npStat
#a.values
np.squeeze(stat1.minCum.compute())

TypeError: opa._updateMin() takes 2 positional arguments but 3 were given

In [16]:
loadPath = "/home/b/b382291/git/data/"
files = os.listdir(loadPath)
files = sorted(files)
savedData = xr.open_dataset(loadPath + files[0])


ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = savedData.meanVar - npMean
a.values


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [17]:
ds2 = tasr.isel(time=slice(24,30)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = meanStat.meanCum - npMean


NameError: name 'tasr' is not defined

In [ ]:
dm.stat

<xarray.DataArray 'stat' (time: 1, lat: 900, lon: 1800)>
dask.array<zeros_like, shape=(1, 900, 1800), dtype=float64, chunksize=(1, 900, 1800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20T23:00:00
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9

In [32]:
stat1.varCum.compute()

array([[[0.61608502, 0.61608502, 0.61608502, ..., 0.61608502,
         0.61608502, 0.61608502],
        [0.56514207, 0.56514207, 0.56514207, ..., 0.56514207,
         0.56514207, 0.56514207],
        [2.3289892 , 2.3289892 , 2.3289892 , ..., 2.3289892 ,
         2.3289892 , 2.3289892 ],
        ...,
        [2.02783125, 2.02783125, 2.02783125, ..., 2.02783125,
         2.02783125, 2.02783125],
        [2.15079509, 2.15079509, 2.15079509, ..., 2.15079509,
         2.15079509, 2.15079509],
        [1.57241274, 1.57241274, 1.57241274, ..., 1.57241274,
         1.57241274, 1.57241274]]])

In [39]:
try:
    getattr(tasr, "var")
except AttributeError:
    b = 4

In [64]:
stat1.statistic
b = str('statistic')
#stat1.__getattr__(b)

#stat1.getattr(stat1, 'statistic')
stat1.variable

'tas'

In [81]:
stat1.stdCum.compute()

array([[[1.73636256, 1.73636256, 1.73636256, ..., 1.73636256,
         1.73636256, 1.73636256],
        [1.71184255, 1.71184255, 1.71184255, ..., 1.71184255,
         1.71184255, 1.71184255],
        [1.66101838, 1.66101838, 1.66101838, ..., 1.66101838,
         1.66101838, 1.66101838],
        ...,
        [0.51507693, 0.51507693, 0.51507693, ..., 0.51507693,
         0.51507693, 0.51507693],
        [0.57755666, 0.57755666, 0.57755666, ..., 0.57755666,
         0.57755666, 0.57755666],
        [0.61978951, 0.61978951, 0.61978951, ..., 0.61978951,
         0.61978951, 0.61978951]]])

In [6]:
# works but really slow 
a = np.array(ds[0,:,:])
b = np.array(ds[1,:,:])

In [5]:
# doesn't work because it's dask 
c = xr.DataArray(ds[1,:,:])
d = xr.DataArray(ds[0,:,:])

In [55]:
# works but really slow 
e = (a < b)
e = e.astype(np.int8)

a[e] = b[e]


In [15]:
# doesn't work 
#f = c < d
#f = f.astype(np.int8)

#np.take(d, f)
#c[f] = d[f]

a = np.array([4, 5, 6, 7],[1,8,9,13])

def _complex_operation(my_arr1, my_arr2):
    f = my_arr1 < my_arr2
    f = f.astype(np.int8)
    my_arr1[f] = my_arr2[f]
    return f 

delayed_result = dask.delayed(_complex_operation)(c, d)
# to create a dask array to use in the future
my_new_arr = da.from_delayed(delayed_result, dtype=c.dtype, shape=c.shape)


TypeError: Field elements must be 2- or 3-tuples, got '1'

In [75]:
c1 = xr.DataArray(np.random.rand(900,1800))
d1 = xr.DataArray(np.random.rand(900,1800))
f1 = (c1 < d1)
f1 = f1.astype(np.int8)
c1[f1] = d1[f1]

In [79]:
c

<xarray.DataArray '10u' (lat: 900, lon: 1800)>
dask.array<getitem, shape=(900, 1800), dtype=float64, chunksize=(900, 1800), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] 2020-01-20T23:00:00
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
Attributes: (12/16)
    paramId:                    165
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    cfVarName:                  u10
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid
    OPA:                        daily_mean_calculated using one-pass algorithm

In [11]:
my_new_arr

dask.array<from-value, shape=(900, 1800), dtype=float64, chunksize=(900, 1800), chunktype=numpy.ndarray>

In [10]:
a = np.array([[[4, 5, 6, 7],[1,10,9,13]],[[0,0,4,0],[0,0,4,0]]])
b = np.array([[[8, 8, 8, 2],[1,7,10,13]],[[0,3,0,1],[0,3,0,1]]])
a = da.from_array(a)
b = da.from_array(b)

a = np.random.rand(1,900,1800)
b = np.random.rand(1,900,1800)
a = da.from_array(a)
b = da.from_array(b)

f = (da.asarray(ds))
e = (da.asarray(stat1.minCum))


def _complex_operation(my_arr1, my_arr2):
    fIndex = my_arr1 < my_arr2
    fIndex = fIndex.astype(np.int8)
    my_arr1[fIndex==0] = my_arr2[fIndex==0]
    return my_arr1 


tempOut = _complex_operation(f,e)

#delayed_result = dask.delayed(_complex_operation)(f,e)
# to create a dask array to use in the future
#my_new_arr = da.from_delayed(delayed_result, dtype=ds.dtype, shape=e.shape)
#cat = my_new_arr.compute()

ValueError: Boolean index assignment in Dask expects equally shaped arrays.
Example: da1[da2] = da3 where da1.shape == (4,), da2.shape == (4,) and da3.shape == (4,).
Alternatively, you can use the extended API that supportsindexing with tuples.
Example: da1[(da2,)] = da3.

In [19]:
e.shape

(1, 900, 1800)

In [20]:
f.shape

(1, 900, 1800)

In [43]:
my_arr1 = (da.asarray(ds))
my_arr2 = (da.asarray(stat1.minCum))
#my_arr1 = np.squeeze(ds[0,0,:])
#my_arr2 = np.squeeze(ds[0,0,:])

#def _complex_operation(my_arr1, my_arr2):
fIndex = my_arr1 < my_arr2
fIndex = fIndex.astype(np.int8)
#my_arr1[(fIndex==0,)] = my_arr2[(fIndex==0,)]
my_arr1[fIndex==0] = my_arr2[fIndex ==0]

#tempOut = _complex_operation(f,e)

In [44]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 240.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32785,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 240.00 GiB
Comm: tcp://127.0.0.1:36153,Total threads: 4
Dashboard: http://127.0.0.1:45055/status,Memory: 240.00 GiB
Nanny: tcp://127.0.0.1:38719,
